## Интерактивен Phi 3 Mini 4K Instruct Chatbot с Whisper

### Въведение:
Интерактивният Phi 3 Mini 4K Instruct Chatbot е инструмент, който позволява на потребителите да взаимодействат с демонстрацията на Microsoft Phi 3 Mini 4K instruct чрез текстов или аудио вход. Чатботът може да се използва за различни задачи, като превод, актуализации за времето и събиране на обща информация.


In [ ]:
#Install required Python Packages
!pip install accelerate
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install flash-attn --no-build-isolation', env={'FLASH_ATTENTION_SKIP_CUDA_BUILD': "TRUE"}, shell=True
!pip install transformers
!pip install wheel
!pip install gradio
!pip install pydub==0.25.1
!pip install edge-tts
!pip install openai-whisper==20231117
!pip install ffmpeg==1.4
# from IPython.display import clear_output
# clear_output()

In [ ]:
# Checking to see if Cuda support is available 
# Output True = Cuda
# Output False = No Cuda (installing Cuda will be required to run the model on GPU)
import os 
import torch
print(torch.cuda.is_available())


Създайте вашия Huggingface Access Token

Създайте нов токен  
Дайте ново име  
Изберете права за запис  
Копирайте токена и го запазете на сигурно място


Следният Python код изпълнява две основни задачи: импортиране на модула `os` и задаване на променлива на средата.

1. Импортиране на модула `os`:
   - Модулът `os` в Python предоставя начин за взаимодействие с операционната система. Той позволява изпълнение на различни задачи, свързани с операционната система, като достъп до променливи на средата, работа с файлове и директории и други.
   - В този код модулът `os` се импортира чрез командата `import`. Тази команда прави функционалността на модула `os` достъпна за използване в текущия Python скрипт.

2. Задаване на променлива на средата:
   - Променлива на средата е стойност, която може да бъде достъпна от програми, работещи на операционната система. Това е начин за съхранение на настройки за конфигурация или друга информация, която може да бъде използвана от множество програми.
   - В този код се задава нова променлива на средата чрез речника `os.environ`. Ключът на речника е `'HF_TOKEN'`, а стойността се задава от променливата `HUGGINGFACE_TOKEN`.
   - Променливата `HUGGINGFACE_TOKEN` е дефинирана точно над този кодов фрагмент и ѝ е присвоена текстова стойност `"hf_**************"` чрез синтаксиса `#@param`. Този синтаксис често се използва в Jupyter notebooks, за да позволи въвеждане от потребителя и конфигуриране на параметри директно в интерфейса на notebook-а.
   - Чрез задаването на променливата на средата `'HF_TOKEN'`, тя може да бъде достъпна от други части на програмата или от други програми, работещи на същата операционна система.

Като цяло, този код импортира модула `os` и задава променлива на средата с име `'HF_TOKEN'` със стойността, предоставена в променливата `HUGGINGFACE_TOKEN`.


In [ ]:
import os
# set the Hugging Face Token from 
# add the Hugging Face Token to the environment variables
HUGGINGFACE_TOKEN = "Enter Hugging Face Key" #@param {type:"string"}
os.environ['HF_TOKEN']HUGGINGFACE_TOKEN

Този кодов фрагмент дефинира функция, наречена clear_output, която се използва за изчистване на изхода на текущата клетка в Jupyter Notebook или IPython. Нека разгледаме кода и разберем неговата функционалност:

Функцията clear_output приема един параметър, наречен wait, който е булева стойност. По подразбиране wait е зададен на False. Този параметър определя дали функцията трябва да изчака, докато новият изход стане достъпен, за да замени съществуващия изход, преди да го изчисти.

Самата функция се използва за изчистване на изхода на текущата клетка. В Jupyter Notebook или IPython, когато клетка генерира изход, като например отпечатан текст или графични диаграми, този изход се показва под клетката. Функцията clear_output позволява да изчистите този изход.

Реализацията на функцията не е предоставена в кодовия фрагмент, както е указано чрез многоточието (...). Многоточието представлява заместител за действителния код, който извършва изчистването на изхода. Реализацията на функцията може да включва взаимодействие с API на Jupyter Notebook или IPython, за да се премахне съществуващият изход от клетката.

Като цяло, тази функция предоставя удобен начин за изчистване на изхода на текущата клетка в Jupyter Notebook или IPython, което улеснява управлението и актуализирането на показания изход по време на интерактивни сесии за програмиране.


In [ ]:
# Download Phi-3-mini-4k-instruct model & Whisper Tiny
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

torch.random.manual_seed(0)

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

#whisper for speech to text()
import whisper
select_model ="tiny" # ['tiny', 'base']
whisper_model = whisper.load_model(select_model)

#from IPython.display import clear_output
#clear_output()

Извършване на текст-към-говор (TTS) с помощта на Edge TTS услугата. Нека разгледаме съответните реализации на функциите една по една:

1. `calculate_rate_string(input_value)`: Тази функция приема входна стойност и изчислява скоростта на гласа за TTS. Входната стойност представлява желаната скорост на речта, където стойност 1 означава нормална скорост. Функцията изчислява скоростта, като изважда 1 от входната стойност, умножава я по 100 и определя знака в зависимост от това дали входната стойност е по-голяма или равна на 1. Функцията връща скоростта във формат "{sign}{rate}".

2. `make_chunks(input_text, language)`: Тази функция приема текст и език като параметри. Тя разделя текста на части според езиково специфични правила. В тази имплементация, ако езикът е "English", функцията разделя текста при всяка точка (".") и премахва водещите и завършващите интервали. След това добавя точка към всяка част и връща филтрирания списък с части.

3. `tts_file_name(text)`: Тази функция генерира име за аудио файла на TTS въз основа на входния текст. Тя извършва няколко трансформации върху текста: премахва завършваща точка (ако има), преобразува текста в малки букви, премахва водещите и завършващите интервали и заменя интервалите с долни черти. След това съкращава текста до максимум 25 символа (ако е по-дълъг) или използва целия текст, ако е празен. Накрая генерира случайна низ с помощта на модула [`uuid`] и го комбинира със съкратения текст, за да създаде име на файла във формат "/content/edge_tts_voice/{truncated_text}_{random_string}.mp3".

4. `merge_audio_files(audio_paths, output_path)`: Тази функция обединява множество аудио файлове в един. Тя приема списък с пътища към аудио файлове и изходен път като параметри. Функцията инициализира празен обект `AudioSegment`, наречен [`merged_audio`]. След това преминава през всеки път към аудио файл, зарежда аудио файла с помощта на метода `AudioSegment.from_file()` от библиотеката `pydub` и добавя текущия аудио файл към обекта [`merged_audio`]. Накрая експортира обединеното аудио към посочения изходен път в MP3 формат.

5. `edge_free_tts(chunks_list, speed, voice_name, save_path)`: Тази функция извършва TTS операция с помощта на Edge TTS услугата. Тя приема списък с текстови части, скорост на речта, име на гласа и път за запазване като параметри. Ако броят на частите е повече от 1, функцията създава директория за съхранение на индивидуалните аудио файлове. След това преминава през всяка част, конструира Edge TTS команда с помощта на функцията `calculate_rate_string()`, името на гласа и текста на частта и изпълнява командата с помощта на функцията `os.system()`. Ако изпълнението на командата е успешно, добавя пътя на генерирания аудио файл към списък. След обработката на всички части, обединява индивидуалните аудио файлове с помощта на функцията `merge_audio_files()` и запазва обединеното аудио към посочения път за запазване. Ако има само една част, директно генерира Edge TTS команда и запазва аудиото към пътя за запазване. Накрая връща пътя за запазване на генерирания аудио файл.

6. `random_audio_name_generate()`: Тази функция генерира случайно име за аудио файл с помощта на модула [`uuid`]. Тя генерира случайно UUID, преобразува го в низ, взема първите 8 символа, добавя разширението ".mp3" и връща случайното име на аудио файла.

7. `talk(input_text)`: Тази функция е основната входна точка за извършване на TTS операция. Тя приема входен текст като параметър. Първо проверява дължината на входния текст, за да определи дали е дълго изречение (по-голямо или равно на 600 символа). Въз основа на дължината и стойността на променливата `translate_text_flag`, определя езика и генерира списък с текстови части с помощта на функцията `make_chunks()`. След това генерира път за запазване на аудио файла с помощта на функцията `random_audio_name_generate()`. Накрая извиква функцията `edge_free_tts()` за извършване на TTS операцията и връща пътя за запазване на генерирания аудио файл.

Като цяло, тези функции работят заедно, за да разделят входния текст на части, генерират име за аудио файла, извършват TTS операция с помощта на Edge TTS услугата и обединяват индивидуалните аудио файлове в един аудио файл.


In [ ]:
#@title Edge TTS
def calculate_rate_string(input_value):
    rate = (input_value - 1) * 100
    sign = '+' if input_value >= 1 else '-'
    return f"{sign}{abs(int(rate))}"


def make_chunks(input_text, language):
    language="English"
    if language == "English":
      temp_list = input_text.strip().split(".")
      filtered_list = [element.strip() + '.' for element in temp_list[:-1] if element.strip() and element.strip() != "'" and element.strip() != '"']
      if temp_list[-1].strip():
          filtered_list.append(temp_list[-1].strip())
      return filtered_list


import re
import uuid
def tts_file_name(text):
    if text.endswith("."):
        text = text[:-1]
    text = text.lower()
    text = text.strip()
    text = text.replace(" ","_")
    truncated_text = text[:25] if len(text) > 25 else text if len(text) > 0 else "empty"
    random_string = uuid.uuid4().hex[:8].upper()
    file_name = f"/content/edge_tts_voice/{truncated_text}_{random_string}.mp3"
    return file_name


from pydub import AudioSegment
import shutil
import os
def merge_audio_files(audio_paths, output_path):
    # Initialize an empty AudioSegment
    merged_audio = AudioSegment.silent(duration=0)

    # Iterate through each audio file path
    for audio_path in audio_paths:
        # Load the audio file using Pydub
        audio = AudioSegment.from_file(audio_path)

        # Append the current audio file to the merged_audio
        merged_audio += audio

    # Export the merged audio to the specified output path
    merged_audio.export(output_path, format="mp3")

def edge_free_tts(chunks_list,speed,voice_name,save_path):
  # print(chunks_list)
  if len(chunks_list)>1:
    chunk_audio_list=[]
    if os.path.exists("/content/edge_tts_voice"):
      shutil.rmtree("/content/edge_tts_voice")
    os.mkdir("/content/edge_tts_voice")
    k=1
    for i in chunks_list:
      print(i)
      edge_command=f'edge-tts  --rate={calculate_rate_string(speed)}% --voice {voice_name} --text "{i}" --write-media /content/edge_tts_voice/{k}.mp3'
      print(edge_command)
      var1=os.system(edge_command)
      if var1==0:
        pass
      else:
        print(f"Failed: {i}")
      chunk_audio_list.append(f"/content/edge_tts_voice/{k}.mp3")
      k+=1
    # print(chunk_audio_list)
    merge_audio_files(chunk_audio_list, save_path)
  else:
    edge_command=f'edge-tts  --rate={calculate_rate_string(speed)}% --voice {voice_name} --text "{chunks_list[0]}" --write-media {save_path}'
    print(edge_command)
    var2=os.system(edge_command)
    if var2==0:
      pass
    else:
      print(f"Failed: {chunks_list[0]}")
  return save_path

# text = "This is Microsoft Phi 3 mini 4k instruct Demo" Simply update the text variable with the text you want to convert to speech
text = 'This is Microsoft Phi 3 mini 4k instruct Demo'  # @param {type: "string"}
Language = "English" # @param ['English']
# Gender of voice simply change from male to female and choose the voice you want to use
Gender = "Female"# @param ['Male', 'Female']
female_voice="en-US-AriaNeural"# @param["en-US-AriaNeural",'zh-CN-XiaoxiaoNeural','zh-CN-XiaoyiNeural']
speed = 1  # @param {type: "number"}
translate_text_flag  = False
if len(text)>=600:
  long_sentence = True
else:
  long_sentence = False

# long_sentence = False # @param {type:"boolean"}
save_path = ''  # @param {type: "string"}
if len(save_path)==0:
  save_path=tts_file_name(text)
if Language == "English" :
  if Gender=="Male":
    voice_name="en-US-ChristopherNeural"
  if Gender=="Female":
    voice_name=female_voice
    # voice_name="en-US-AriaNeural"


if translate_text_flag:
  input_text=text
  # input_text=translate_text(text, Language)
  # print("Translateting")
else:
  input_text=text
if long_sentence==True and translate_text_flag==True:
  chunks_list=make_chunks(input_text,Language)
elif long_sentence==True and translate_text_flag==False:
  chunks_list=make_chunks(input_text,"English")
else:
  chunks_list=[input_text]
# print(chunks_list)
# edge_save_path=edge_free_tts(chunks_list,speed,voice_name,save_path)
# from IPython.display import clear_output
# clear_output()
# from IPython.display import Audio
# Audio(edge_save_path, autoplay=True)

from IPython.display import clear_output
from IPython.display import Audio
if not os.path.exists("/content/audio"):
    os.mkdir("/content/audio")
import uuid
def random_audio_name_generate():
  random_uuid = uuid.uuid4()
  audio_extension = ".mp3"
  random_audio_name = str(random_uuid)[:8] + audio_extension
  return random_audio_name
def talk(input_text):
  global translate_text_flag,Language,speed,voice_name
  if len(input_text)>=600:
    long_sentence = True
  else:
    long_sentence = False

  if long_sentence==True and translate_text_flag==True:
    chunks_list=make_chunks(input_text,Language)
  elif long_sentence==True and translate_text_flag==False:
    chunks_list=make_chunks(input_text,"English")
  else:
    chunks_list=[input_text]
  save_path="/content/audio/"+random_audio_name_generate()
  edge_save_path=edge_free_tts(chunks_list,speed,voice_name,save_path)
  return edge_save_path


edge_save_path=talk(text)
Audio(edge_save_path, autoplay=True)

Реализация на две функции: convert_to_text и run_text_prompt, както и декларация на два класа: str и Audio.

Функцията convert_to_text приема audio_path като вход и транскрибира аудиото в текст, използвайки модел, наречен whisper_model. Функцията първо проверява дали флагът gpu е зададен на True. Ако е така, whisper_model се използва с определени параметри като word_timestamps=True, fp16=True, language='English' и task='translate'. Ако флагът gpu е False, whisper_model се използва с fp16=False. Получената транскрипция след това се записва във файл с име 'scan.txt' и се връща като текст.

Функцията run_text_prompt приема съобщение и chat_history като вход. Тя използва функцията phi_demo, за да генерира отговор от чатбот въз основа на входното съобщение. Генерираният отговор след това се предава на функцията talk, която конвертира отговора в аудио файл и връща пътя до файла. Класът Audio се използва за показване и възпроизвеждане на аудио файла. Аудиото се показва чрез функцията display от модула IPython.display, а обектът Audio се създава с параметъра autoplay=True, така че аудиото да започне да се възпроизвежда автоматично. chat_history се актуализира с входното съобщение и генерирания отговор, след което се връщат празен низ и актуализираният chat_history.

Класът str е вграден клас в Python, който представлява последователност от символи. Той предоставя различни методи за манипулиране и работа със стрингове, като например capitalize, casefold, center, count, encode, endswith, expandtabs, find, format, index, isalnum, isalpha, isascii, isdecimal, isdigit, isidentifier, islower, isnumeric, isprintable, isspace, istitle, isupper, join, ljust, lower, lstrip, partition, replace, removeprefix, removesuffix, rfind, rindex, rjust, rpartition, rsplit, rstrip, split, splitlines, startswith, strip, swapcase, title, translate, upper, zfill и други. Тези методи позволяват извършване на операции като търсене, заместване, форматиране и манипулиране на стрингове.

Класът Audio е персонализиран клас, който представлява аудио обект. Той се използва за създаване на аудио плейър в средата на Jupyter Notebook. Класът приема различни параметри като data, filename, url, embed, rate, autoplay и normalize. Параметърът data може да бъде numpy масив, списък от семпли, низ, представляващ име на файл или URL, или сурови PCM данни. Параметърът filename се използва за посочване на локален файл, от който да се заредят аудио данните, а параметърът url се използва за посочване на URL, от който да се изтеглят аудио данните. Параметърът embed определя дали аудио данните трябва да бъдат вградени чрез data URI или да се реферират от оригиналния източник. Параметърът rate указва честотата на семплиране на аудио данните. Параметърът autoplay определя дали аудиото трябва да започне да се възпроизвежда автоматично. Параметърът normalize указва дали аудио данните трябва да бъдат нормализирани (мащабирани) до максималния възможен обхват. Класът Audio също предоставя методи като reload за презареждане на аудио данните от файл или URL, както и атрибути като src_attr, autoplay_attr и element_id_attr за извличане на съответните атрибути за аудио елемента в HTML.

Като цяло, тези функции и класове се използват за транскрибиране на аудио в текст, генериране на аудио отговори от чатбот и показване и възпроизвеждане на аудио в средата на Jupyter Notebook.


In [ ]:
#@title Run gradio app
def convert_to_text(audio_path):
  gpu=True
  if gpu:
    result = whisper_model.transcribe(audio_path,word_timestamps=True,fp16=True,language='English',task='translate')
  else:
    result = whisper_model.transcribe(audio_path,word_timestamps=True,fp16=False,language='English',task='translate')
  with open('scan.txt', 'w') as file:
    file.write(str(result))
  return result["text"]


import gradio as gr
from IPython.display import Audio, display
def run_text_prompt(message, chat_history):
    bot_message = phi_demo(message)
    edge_save_path=talk(bot_message)
    # print(edge_save_path)
    display(Audio(edge_save_path, autoplay=True))

    chat_history.append((message, bot_message))
    return "", chat_history


def run_audio_prompt(audio, chat_history):
    if audio is None:
        return None, chat_history
    print(audio)
    message_transcription = convert_to_text(audio)
    _, chat_history = run_text_prompt(message_transcription, chat_history)
    return None, chat_history


with gr.Blocks() as demo:
    chatbot = gr.Chatbot(label="Chat with Phi 3 mini 4k instruct")

    msg = gr.Textbox(label="Ask anything")
    msg.submit(run_text_prompt, [msg, chatbot], [msg, chatbot])

    with gr.Row():
        audio = gr.Audio(sources="microphone", type="filepath")

        send_audio_button = gr.Button("Send Audio", interactive=True)
        send_audio_button.click(run_audio_prompt, [audio, chatbot], [audio, chatbot])

demo.launch(share=True,debug=True)


---

**Отказ от отговорност**:  
Този документ е преведен с помощта на AI услуга за превод [Co-op Translator](https://github.com/Azure/co-op-translator). Въпреки че се стремим към точност, моля, имайте предвид, че автоматичните преводи може да съдържат грешки или неточности. Оригиналният документ на неговия изходен език трябва да се счита за авторитетен източник. За критична информация се препоръчва професионален превод от човек. Ние не носим отговорност за каквито и да е недоразумения или погрешни интерпретации, произтичащи от използването на този превод.
